## Create Dam Catchment Shapefiles

## Setup
### Import packages

In [2]:
import pcraster as pcr
import gdal, gdalconst
import numpy as np
import os
import geopandas as gpd
import matplotlib.pyplot as plt 
import pandas as pd
from shapely import wkt
from shapely.geometry import shape

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings('ignore')

### Set data directory

In [4]:
os.getcwd()
# dataDir = os.path.join(r'/scratch/gpfs/tianboz/GlobalDams/Data')
dataDir = os.path.join(r'/Users/AliceZhang/Dropbox/Research_Columbia/Zhang_Gu_Conflict_Dam/Data/Raw')

'/Users/AliceZhang/Dropbox/Research_Columbia/1_Data_Collection/GlobalDam/Analysis/codePython'

### Get shapefile geographic extent

In [8]:
os.chdir(dataDir)
gdat_af = gpd.read_file("CorrectCoord_GDAT_HydroRivers/CorrectCoord_GDAT_HydroRivers_Africa.shp")
gdat_na = gpd.read_file("CorrectCoord_GDAT_HydroRivers/CorrectCoord_GDAT_HydroRivers_NorthAmerica.shp")
gdat_sa = gpd.read_file("CorrectCoord_GDAT_HydroRivers/CorrectCoord_GDAT_HydroRivers_SouthAmerica.shp")
gdat_as_eu_au = gpd.read_file("CorrectCoord_GDAT_HydroRivers/CorrectCoord_GDAT_HydroRivers_AsiaOceaniaEurope.shp")
gdat_world = gpd.read_file("CorrectCoord_GDAT_HydroRivers/CorrectCoord_GDAT_HydroRivers_World.shp")

In [9]:
print("Tabulation of GDAT observations in AF")
pd.crosstab(gdat_af['Continent'], columns='count')

print("Tabulation of GDAT observations in NA")
pd.crosstab(gdat_na['Continent'], columns='count')

print("Tabulation of GDAT observations in SA")
pd.crosstab(gdat_sa['Continent'], columns='count')

print("Tabulation of GDAT observations in AS+EU+AU")
pd.crosstab(gdat_as_eu_au['Continent'], columns='count')

Tabulation of GDAT observations in AF


col_0,count
Continent,
Africa,5778
Europe,18


Tabulation of GDAT observations in NA


col_0,count
Continent,
North America,8006
Oceania,1
South America,1


Tabulation of GDAT observations in SA


col_0,count
Continent,
South America,5729


Tabulation of GDAT observations in AS+EU+AU


col_0,count
Continent,
Asia,8776
Europe,3116
Oceania,266
South America,1


In [10]:
# Reference extent: http://www.fao.org/geonetwork/srv/en/metadata.show?id=37341

print("Boundary for AF")
gdat_af.total_bounds
# Boundary for Africa: -25.16517, -34.67361111, 51.10000, 38.20000 

print("Boundary for NA")
gdat_na.total_bounds
# Boundary for North America: -162.873355, 8.44283333, -52.7123, 68.073355

print("Boundary for SA")
gdat_sa.total_bounds
# Boundary for South America: -80.84525591, -43.70208333, -34.80208333, 11.34375

print("Boundary for AS+EU+AU")
gdat_as_eu_au.total_bounds
# Boundary for Asia + Europe + Australia: -60.93625, -45.8816, 177.37291667, 70.38541667

print("Boundary for GDAT global")
gdat_world.total_bounds
# Boundary for GDAT global: -162.873355, -45.8816, 177.37291667, 70.38541667



Boundary for AF


array([-25.16517   , -34.67361111,  49.25      ,  37.23958333])

Boundary for NA


array([-162.873355  ,    8.44283333,  -52.7123    ,   68.073355  ])

Boundary for SA


array([-80.84525591, -43.70208333, -34.80208333,  11.34375   ])

Boundary for AS+EU+AU


array([-60.93625   , -45.8816    , 177.37291667,  70.38541667])

Boundary for GDAT global


array([-162.873355  ,  -45.8816    ,  177.37291667,   70.38541667])

## Use PCRaster to extract dam catchment area

### Step 0
- PCRaster reference: http://karssenberg.geo.uu.nl/labs/dataProcessingDem.html

- Convert dam .shp file into raster .tif file in shell

    - Directory

    `cd /scratch/gpfs/tianboz/GlobalDams/Data/CorrectCoord_GDAT_HydroRivers`
    
    - Use the value "1" for all dams
    
     `gdal_rasterize -l GRanD_dams_v1_1 -burn 1 -tr 0.0174 0.0174 GRanD_dams_v1_1.shp GRanD_dams_v1_1.tif`
    
    - Use featureID as the value for each dam
    
    `gdal_rasterize -l GlobalDam_v0_abbVarName -a Feature_ID -tr 0.0174 0.0174 GlobalDam_v0_abbVarName.shp GlobalDam_v0_abbVarName.tif`
    
    `gdal_rasterize -l CorrectCoord_GDAT_HydroRivers_Africa -a Feature_ID -tr 0.0174532925199433 0.0174532925199433 CorrectCoord_GDAT_HydroRivers_Africa.shp CorrectCoord_GDAT_HydroRivers_Africa.tif`
    
    `gdal_rasterize -l CorrectCoord_GDAT_HydroRivers_NorthAmerica -a Feature_ID -tr 0.0174532925199433 0.0174532925199433 CorrectCoord_GDAT_HydroRivers_NorthAmerica.shp CorrectCoord_GDAT_HydroRivers_NorthAmerica.tif`
    
    `gdal_rasterize -l CorrectCoord_GDAT_HydroRivers_SouthAmerica -a Feature_ID -tr 0.0174532925199433 0.0174532925199433 CorrectCoord_GDAT_HydroRivers_SouthAmerica.shp CorrectCoord_GDAT_HydroRivers_SouthAmerica.tif`
    
    `gdal_rasterize -l CorrectCoord_GDAT_HydroRivers_AsiaOceaniaEurope -a Feature_ID -tr 0.0174532925199433 0.0174532925199433 CorrectCoord_GDAT_HydroRivers_AsiaOceaniaEurope.shp CorrectCoord_GDAT_HydroRivers_AsiaOceaniaEurope.tif`
    
    `gdal_rasterize -l CorrectCoord_GDAT_HydroRivers_World -a Feature_ID -tr 0.0174532925199433 0.0174532925199433 CorrectCoord_GDAT_HydroRivers_World.shp CorrectCoord_GDAT_HydroRivers_World.tif`
    
    
- Convert .tif file into PCRaster .map file in shell
    
    `gdal_translate -ot Int32 -of PCRaster -mo PCRASTER_VALUESCALE=VS_SCALAR CorrectCoord_GDAT_HydroRivers_Africa.tif CorrectCoord_GDAT_HydroRivers_Africa.map`
    
     `gdal_translate -ot Int32 -of PCRaster -mo PCRASTER_VALUESCALE=VS_SCALAR CorrectCoord_GDAT_HydroRivers_NorthAmerica.tif CorrectCoord_GDAT_HydroRivers_NorthAmerica.map`
     
     `gdal_translate -ot Int32 -of PCRaster -mo PCRASTER_VALUESCALE=VS_SCALAR CorrectCoord_GDAT_HydroRivers_SouthAmerica.tif CorrectCoord_GDAT_HydroRivers_SouthAmerica.map`
      
     `gdal_translate -ot Int32 -of PCRaster -mo PCRASTER_VALUESCALE=VS_SCALAR CorrectCoord_GDAT_HydroRivers_AsiaOceaniaEurope.tif CorrectCoord_GDAT_HydroRivers_AsiaOceaniaEurope.map`
   
     `gdal_translate -ot Int32 -of PCRaster -mo PCRASTER_VALUESCALE=VS_SCALAR CorrectCoord_GDAT_HydroRivers_World.tif CorrectCoord_GDAT_HydroRivers_World.map`
   
   
- Find the GDAT file size in pixels and lines

    `gdalinfo CorrectCoord_GDAT_HydroRivers_Africa.tif`
    
    `gdalinfo CorrectCoord_GDAT_HydroRivers_NorthAmerica.tif`
    
    `gdalinfo CorrectCoord_GDAT_HydroRivers_SouthAmerica.tif`
    
    `gdalinfo CorrectCoord_GDAT_HydroRivers_AsiaOceaniaEurope.tif`
    
    `gdalinfo CorrectCoord_GDAT_HydroRivers_World.tif`
    
    
- Clip DEM .bil file to GDAT .shp file extent

    `cd /scratch/gpfs/tianboz/GlobalDams/Data/Elevation/HydroSHEDS/DEM_30s_BIL`
    
    `gdalwarp -te -25.16517 -34.67361111 49.250000 37.23958333 -ts 4265 4121 af_dem_30s_bil/af_dem_30s.bil af_dem_30s_bil/af_dem_30s_subset_gdat.bil`
        
    `gdalwarp -te -162.873355 8.44283333 -52.7123 68.073355 -ts 6313 3418 na_ca_dem_30s_bil/na_ca_dem_30s.bil na_ca_dem_30s_bil/na_ca_dem_30s_subset_gdat.bil`
    
    `gdalwarp -te -80.84525591 -43.70208333 -34.80208333 11.34375 -ts 2639 3155 sa_dem_30s_bil/sa_dem_30s.bil sa_dem_30s_bil/sa_dem_30s_subset_gdat.bil`

    `cd /scratch/gpfs/tianboz/GlobalDams/Data/Elevation/GMTED2010`
    
    `gdalwarp -te -60.93625 -45.8816 177.37291667 70.38541667 -ts 13655 6663 as_eu_au_dem_30s_bil/as_eu_au_dem_30s.bil as_eu_au_dem_30s_bil/as_eu_au_dem_30s_subset_gdat.bil`

    `gdalwarp -te -162.873355 -45.8816 177.37291667 70.38541667 -ts 19496 6663 mn75_grd/mn75.tif mn75_grd/mn75_subset_gdat.tif`
    
    
- Convert DEM .bil file into PCRaster .map file in shell

    `gdal_translate -ot Float32 -of PCRaster -mo PCRASTER_VALUESCALE=VS_SCALAR af_dem_30s_bil/af_dem_30s_subset_gdat.bil af_dem_30s_bil/af_dem_30s_subset_gdat.map`
    
    `gdal_translate -ot Float32 -of PCRaster -mo PCRASTER_VALUESCALE=VS_SCALAR na_ca_dem_30s_bil/na_ca_dem_30s_subset_gdat.bil na_ca_dem_30s_bil/na_ca_dem_30s_subset_gdat.map`
    
    `gdal_translate -ot Float32 -of PCRaster -mo PCRASTER_VALUESCALE=VS_SCALAR sa_dem_30s_bil/sa_dem_30s_subset_gdat.bil sa_dem_30s_bil/sa_dem_30s_subset_gdat.map`
    
    `gdal_translate -ot Float32 -of PCRaster -mo PCRASTER_VALUESCALE=VS_SCALAR as_eu_au_dem_30s_bil/as_eu_au_dem_30s_subset_gdat.bil as_eu_au_dem_30s_bil/as_eu_au_dem_30s_subset_gdat.map`
    
    `gdal_translate -ot Float32 -of PCRaster -mo PCRASTER_VALUESCALE=VS_SCALAR mn75_grd/mn75_subset_gdat.tif mn75_grd/mn75_subset_gdat.map`
    
    

### Merge DEM files to match GDAT continents

cd /scratch/gpfs/tianboz/GlobalDams/Data/Elevation/HydroSHEDS/DEM_30s_BIL/

gdal_merge.py -o as_eu_au_dem_30s.bil as_dem_30s_bil/as_dem_30s.bil eu_dem_30s_bil/eu_dem_30s.bil au_dem_30s_bil/au_dem_30s.bil 

gdal_merge.py -o na_ca_dem_30s.bil na_dem_30s_bil/na_dem_30s.bil ca_dem_30s_bil/ca_dem_30s.bil

### Convert file formats

cd /scratch/gpfs/tianboz/GlobalDams/Data/CorrectCoord_GDAT_HydroRivers

gdal_rasterize -l CorrectCoord_GDAT_HydroRivers_Africa -a Feature_ID -tr 0.0174532925199433 0.0174532925199433 CorrectCoord_GDAT_HydroRivers_Africa.shp CorrectCoord_GDAT_HydroRivers_Africa.tif

gdal_rasterize -l CorrectCoord_GDAT_HydroRivers_North -a Feature_ID -tr 0.0174532925199433 0.0174532925199433 CorrectCoord_GDAT_HydroRivers_Africa.shp CorrectCoord_GDAT_HydroRivers_Africa.tif



gdal_translate -ot Int32 -of PCRaster -mo PCRASTER_VALUESCALE=VS_SCALAR CorrectCoord_GDAT_HydroRivers_Africa.tif CorrectCoord_GDAT_HydroRivers_Africa.map

cd /scratch/gpfs/tianboz/GlobalDams/Data/Elevation/HydroSHEDS/DEM_30s_BIL/

gdalwarp -te -25.16517 -34.67361111 49.250000 37.23958333 -ts 4265 4121 af_dem_30s_bil/af_dem_30s.bil af_dem_30s_bil/af_dem_30s_subset_gdat.bil -overwrite

gdal_translate -ot Float32 -of PCRaster -mo PCRASTER_VALUESCALE=VS_SCALAR af_dem_30s_bil/af_dem_30s_subset_gdat.bil af_dem_30s_bil/af_dem_30s_subset_gdat.map



### Step 1

- Load and plot DEM file

### Step 2
- Calculate Local Drainage Direction (LDD), which indicates the direction of flow of material (e.g. water or sediment) from one cell to its immediate steepest down slope neighboring cell, using the PCRaster lddcreate operator.

### Step 3
- Verify and relocate to make sure that dams snap to the correct river network

### Step 4
- Calculate upstream catchment area of dams using PCRaster operator subcatchment
    (need to make sure dam location raster has the same number of cells as the elevation raster; same spatial extent)
  

In [11]:
# World
os.chdir(os.path.join(dataDir, "Elevation/GMTED2010/mn75_grd/"))
pcr.setclone("mn75_subset_gdat.map") 

print("Step: Calculate Local Drainage Direction (LDD)")
world_ldd = pcr.lddcreate("mn75_subset_gdat.map", 1e31, 1e31, 1e31, 1e31)
pcr.report(world_ldd, "world_ldd.map")

print("Step: Calculate upstream catchment area of dams using PCRaster operator subcatchment")
os.chdir(os.path.join(dataDir,"CorrectCoord_GDAT_HydroRivers"))
pcr.setclone("CorrectCoord_GDAT_HydroRivers_World.map") 
gdat_world = pcr.readmap("CorrectCoord_GDAT_HydroRivers_World.map")

gdat_catchment_world = pcr.subcatchment(world_ldd, gdat_world)
os.chdir(os.path.join(dataDir, "PCRaster_Catchment"))
pcr.report(gdat_catchment_world, "GDAT_Catchment_World.map")

Step: Calculate Local Drainage Direction (LDD)
Step: Calculate upstream catchment area of dams using PCRaster operator subcatchment


In [10]:
os.chdir(os.path.join(dataDir, "Elevation/GMTED2010/mn75_grd/"))
pcr.setclone("mn75_subset_gdat.map") 
pcr.aguila("mn75_subset_gdat.map","world_ldd.map")

In [5]:
# Africa
os.chdir(os.path.join(dataDir, "Elevation/HydroSHEDS/DEM_30s_BIL/af_dem_30s_bil/"))
pcr.setclone("af_dem_30s_subset_gdat.map") 

print("Step: Load DEM file")
af_dem = pcr.readmap("af_dem_30s_subset_gdat.map")

print("Step: Calculate Local Drainage Direction (LDD)")
af_ldd = pcr.lddcreate("af_dem_30s_subset_gdat.map", 1e31, 1e31, 1e31, 1e31)
pcr.report(af_ldd, "af_ldd.map")

print("Step: Calculate upstream catchment area of dams using PCRaster operator subcatchment")
os.chdir(os.path.join(dataDir,"CorrectCoord_GDAT_HydroRivers"))
pcr.setclone("CorrectCoord_GDAT_HydroRivers_Africa.map") 
gdat_af = pcr.readmap("CorrectCoord_GDAT_HydroRivers_Africa.map")

gdat_catchment_af = pcr.subcatchment(af_ldd, gdat_af)
os.chdir(os.path.join(dataDir, "PCRaster_Catchment"))
pcr.report(gdat_catchment_af, "GDAT_Catchment_Africa.map")

Step: Load DEM file
Step: Calculate Local Drainage Direction (LDD)
Step: Calculate upstream catchment area of dams using PCRaster operator subcatchment


In [ ]:
# North America
os.chdir(os.path.join(dataDir, "Elevation/HydroSHEDS/DEM_30s_BIL/na_ca_dem_30s_bil/"))
pcr.setclone("na_ca_dem_30s_subset_gdat.map") 

print("Step: Load DEM file")
na_dem = pcr.readmap("na_ca_dem_30s_subset_gdat.map")

print("Step: Calculate Local Drainage Direction (LDD)")
na_ldd = pcr.lddcreate("na_ca_dem_30s_subset_gdat.map", 1e31, 1e31, 1e31, 1e31)
pcr.report(na_ldd, "na_ca_ldd.map")

print("Step: Calculate upstream catchment area of dams using PCRaster operator subcatchment")
os.chdir(os.path.join(dataDir,"CorrectCoord_GDAT_HydroRivers"))
pcr.setclone("CorrectCoord_GDAT_HydroRivers_NorthAmerica.map") 
gdat_na = pcr.readmap("CorrectCoord_GDAT_HydroRivers_NorthAmerica.map")

gdat_catchment_na = pcr.subcatchment(na_ldd, gdat_na)
os.chdir(os.path.join(dataDir, "PCRaster_Catchment"))
pcr.report(gdat_catchment_na, "GDAT_Catchment_NorthAmerica.map")

In [ ]:
# South America
os.chdir(os.path.join(dataDir, "Elevation/HydroSHEDS/DEM_30s_BIL/sa_dem_30s_bil/"))
pcr.setclone("sa_dem_30s_subset_gdat.map") 

print("Step: Load DEM file")
sa_dem = pcr.readmap("sa_dem_30s_subset_gdat.map")

print("Step: Calculate Local Drainage Direction (LDD)")
sa_ldd = pcr.lddcreate("sa_dem_30s_subset_gdat.map", 1e31, 1e31, 1e31, 1e31)
pcr.report(sa_ldd, "sa_ldd.map")

print("Step: Calculate upstream catchment area of dams using PCRaster operator subcatchment")
os.chdir(os.path.join(dataDir,"CorrectCoord_GDAT_HydroRivers"))
pcr.setclone("CorrectCoord_GDAT_HydroRivers_SouthAmerica.map") 
gdat_sa = pcr.readmap("CorrectCoord_GDAT_HydroRivers_SouthAmerica.map")

gdat_catchment_sa = pcr.subcatchment(sa_ldd, gdat_sa)
os.chdir(os.path.join(dataDir, "PCRaster_Catchment"))
pcr.report(gdat_catchment_sa, "GDAT_Catchment_SouthAmerica.map")

In [ ]:
# Asia + Europe + Oceania
os.chdir(os.path.join(dataDir, "Elevation/HydroSHEDS/DEM_30s_BIL/as_eu_au_dem_30s_bil/"))
pcr.setclone("as_eu_au_dem_30s_subset_gdat.map") 

print("Step: Load DEM file")
as_eu_au_dem = pcr.readmap("as_eu_au_dem_30s_subset_gdat.map")

print("Step: Calculate Local Drainage Direction (LDD)")
as_eu_au_ldd = pcr.lddcreate("as_eu_au_dem_30s_subset_gdat.map", 1e31, 1e31, 1e31, 1e31)
pcr.report(as_eu_au_ldd, "as_eu_au_ldd.map")

print("Step: Calculate upstream catchment area of dams using PCRaster operator subcatchment")
os.chdir(os.path.join(dataDir,"CorrectCoord_GDAT_HydroRivers"))
pcr.setclone("CorrectCoord_GDAT_HydroRivers_AsiaOceaniaEurope.map") 
gdat_as_eu_au = pcr.readmap("CorrectCoord_GDAT_HydroRivers_AsiaOceaniaEurope.map")

gdat_catchment_as_eu_au = pcr.subcatchment(as_eu_au_ldd, gdat_as_eu_au)
os.chdir(os.path.join(dataDir, "PCRaster_Catchment"))
pcr.report(gdat_catchment_as_eu_au, "GDAT_Catchment_AsiaOceaniaEurope.map")

### Step 5
- Convert output to shapefile in shell

    `cd /scratch/gpfs/tianboz/GlobalDams/Data/PCRaster_Catchment`
    
    `gdal_translate GDAT_Catchment_Africa.map GDAT_Catchment_Africa.tif`
    
    `gdal_polygonize.py GDAT_Catchment_Africa.tif GDAT_Catchment_Africa.shp`
    
    `gdal_translate GDAT_Catchment_NorthAmerica.map GDAT_Catchment_NorthAmerica.tif`
    
    `gdal_polygonize.py GDAT_Catchment_NorthAmerica.tif GDAT_Catchment_NorthAmerica.shp`

    `gdal_translate GDAT_Catchment_SouthAmerica.map GDAT_Catchment_SouthAmerica.tif`
    
    `gdal_polygonize.py GDAT_Catchment_SouthAmerica.tif GDAT_Catchment_SouthAmerica.shp`

    `gdal_translate GDAT_Catchment_AsiaOceaniaEurope.map GDAT_Catchment_AsiaOceaniaEurope.tif`
    
    `gdal_polygonize.py GDAT_Catchment_AsiaOceaniaEurope.tif GDAT_Catchment_AsiaOceaniaEurope.shp`
    
    `gdal_translate GDAT_Catchment_World.map GDAT_Catchment_World.tif` 
    
    `gdal_polygonize.py GDAT_Catchment_World.tif GDAT_Catchment_World.shp`

### Step 6: Merge catchment with GDAT dam layer to get attributes

In [3]:
# # Combine continent-specific catchment shapefiles
# os.chdir(os.path.join(dataDir, "PCRaster_Catchment"))
# catchment_af = gpd.read_file("GDAT_Catchment_Africa.shp")
# catchment_na = gpd.read_file("GDAT_Catchment_NorthAmerica.shp")
# catchment_sa = gpd.read_file("GDAT_Catchment_SouthAmerica.shp")
# catchment_as_eu_au = gpd.read_file("GDAT_Catchment_AsiaOceaniaEurope.shp")
# catchment_world = catchment_af.append(catchment_na).append(catchment_sa).append(catchment_as_eu_au)
# catchment_world.to_file("GDAT_Catchment_HydroSHEDS_World.shp")

In [4]:
# Import catchment 
os.chdir(os.path.join(dataDir, "PCRaster_Catchment"))
catchment_world = gpd.read_file("GDAT_Catchment_World.shp")

In [5]:
# Remove 0 DN values 
catchment_world.head()
catchment_world['DN'].value_counts()
catchment_world_rmDN = catchment_world[catchment_world.DN != 0]
catchment_world_rmDN.rename(columns = {'DN':'Feature_ID'}, inplace = True)
catchment_world_rmDN.head()
catchment_world_rmDN['Feature_ID'].value_counts()

,DN,geometry
0,0,"POLYGON ((-160.82259 70.39414, -160.80514 70.3..."
1,0,"POLYGON ((-151.41527 70.39414, -151.39782 70.3..."
2,0,"POLYGON ((-128.21984 70.39414, -128.20239 70.3..."
3,0,"POLYGON ((-128.18494 70.39414, -128.09767 70.3..."
4,0,"POLYGON ((-92.03917 70.39414, -92.02171 70.394..."


0        18872
6046        17
23048       11
20737        8
2562         6
         ...  
2004         1
6102         1
8151         1
20445        1
18457        1
Name: DN, Length: 28461, dtype: int64

,Feature_ID,geometry
8,33769,"POLYGON ((26.92247 70.39414, 27.00974 70.39414..."
19,31950,"POLYGON ((26.80030 70.37669, 26.83521 70.37669..."
202,32901,"POLYGON ((29.08668 70.02762, 29.12159 70.02762..."
306,33568,"POLYGON ((21.77375 69.71346, 21.79121 69.71346..."
317,35033,"POLYGON ((23.65871 69.69601, 23.67616 69.69601..."


6046     17
23048    11
20737     8
2159      6
2562      6
         ..
2004      1
6102      1
8151      1
20445     1
4098      1
Name: Feature_ID, Length: 28460, dtype: int64

In [6]:
# Dissolve polygons with the same FeatureID
catchment_world_rmDN = catchment_world_rmDN.dissolve(by='Feature_ID', aggfunc = 'first').reset_index()

In [7]:
# Save global dam catchment shapefile with DN = 0 removed and polygons dissolved
os.chdir(os.path.join(dataDir, "PCRaster_Catchment"))
catchment_world_rmDN.to_file("GDAT_Catchment_World_Cleaned.shp")

In [8]:
# Load GDAT world data
os.chdir(dataDir)
gdat_world = gpd.read_file("CorrectCoord_GDAT_HydroRivers/CorrectCoord_GDAT_HydroRivers_World.shp")

# Replace geometry as string
gdat_world['geometry'] = gdat_world['geometry'].apply(lambda x: wkt.dumps(x))
gdat_world.rename(columns = {'geometry': 'Correct_Coord'}, inplace = True)

In [9]:
# Merge catchment with GDAT file for attributes
gdat_catchment_world = gdat_world.merge(catchment_world_rmDN, on = 'Feature_ID', how='outer', validate="one_to_one")

In [10]:
# Save merged GDAT & catchment data
os.chdir(dataDir)
gdat_catchment_world.to_file("PCRaster_Catchment/GDAT_Catchment_World_Merged.shp")

#### Other Useful Code 

In [ ]:
# Read .bil file 
def readbil(bil):
    gdal.GetDriverByName('EHdr').Register()
    img = gdal.Open(bil)
    band = img.GetRasterBand(1)
    data = band.ReadAsArray()
    return data
af = readbil(os.path.join(dataDir, "Raw/Elevation/HydroSHEDS/DEM_15s_BIL/af_dem_15s_bil", "af_dem_15s.bil"))
af.dtype

In [ ]:
pcr.aguila?